In [2]:
import qiskit
qiskit.__version__

'1.1.1'

In [3]:
from qiskit.circuit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit.classical import expr, types
 
qr = QuantumRegister(2, "q")
cr = ClassicalRegister(2, "c")
qc = QuantumCircuit(qr, cr)
# Add two input variables to the circuit with different types.
a = qc.add_input("a", types.Bool())
mask = qc.add_input("mask", types.Uint(2))
 
# Test whether the input variable was true at runtime.
with qc.if_test(a) as else_:
    qc.x(0)
with else_:
    qc.h(0)
 
qc.cx(0, 1)
qc.measure(qr, cr)
 
# Add a typed variable manually, initialized to the same value as the classical register.
b = qc.add_var("b", expr.lift(cr))
 
qc.reset([0, 1])
qc.h(0)
qc.cx(0, 1)
qc.measure(qr, cr)
 
# Store some calculated value into the `b` variable.
qc.store(b, expr.bit_and(b, cr))
# Test whether we had equality, up to a mask.
with qc.if_test(expr.equal(expr.bit_and(b, mask), mask)):
    qc.x(0)

In [4]:
import numpy as np
from qiskit import transpile
from qiskit.circuit.library import IQP
from qiskit.primitives import BackendEstimatorV2
from qiskit.providers.fake_provider import Fake7QPulseV1
from qiskit.quantum_info import SparsePauliOp, random_hermitian
 
backend = Fake7QPulseV1()
estimator = BackendEstimatorV2(backend=backend)
n_qubits = 5
mat = np.real(random_hermitian(n_qubits, seed=1234))
circuit = IQP(mat)
observable = SparsePauliOp("Z" * n_qubits)
isa_circuit = transpile(circuit, backend=backend, optimization_level=1)
isa_observable = observable.apply_layout(isa_circuit.layout)
job = estimator.run([(isa_circuit, isa_observable)], precision=0.01)
result = job.result()
print(f"> Expectation value: {result[0].data.evs}")
print(f"> Standard error: {result[0].data.stds}")
print(f"> Metadata: {result[0].metadata}")

> Expectation value: 0.0674
> Standard error: 0.009977260345405446
> Metadata: {'target_precision': 0.01}


c:\Users\malej\miniconda3\envs\cwq\Lib\site-packages\qiskit\providers\fake_provider\fake_backend.py:153: RuntimeWarning: Aer not found using BasicAer and no noise
  warnings.warn("Aer not found using BasicAer and no noise", RuntimeWarning)


In [7]:
import numpy as np
from qiskit import transpile
from qiskit.circuit.library import IQP
from qiskit.primitives import BackendSamplerV2
from qiskit.providers.fake_provider import Fake7QPulseV1
from qiskit.quantum_info import random_hermitian
 
backend = Fake7QPulseV1()
sampler = BackendSamplerV2(backend=backend)
n_qubits = 5
mat = np.real(random_hermitian(n_qubits, seed=1234))
circuit = IQP(mat)
circuit.measure_all()
isa_circuit = transpile(circuit, backend=backend, optimization_level=1)
job = sampler.run([isa_circuit], shots=100)
result = job.result()
print(f"> bitstrings: {result[0].data.meas.get_bitstrings()}")
print(f"> counts: {result[0].data.meas.get_counts()}")
print(f"> Metadata: {result[0].metadata}")

> bitstrings: ['01000', '00000', '00001', '00001', '00000', '01010', '01011', '00000', '00000', '00000', '00100', '01011', '00100', '01011', '01000', '00010', '01010', '01011', '11000', '00001', '00010', '00000', '00001', '01010', '11010', '01010', '11101', '01000', '00001', '00000', '00001', '01110', '01010', '11000', '01011', '00000', '00010', '00011', '01010', '00010', '00010', '00100', '11000', '11010', '00000', '00000', '00011', '01000', '11000', '00000', '00011', '00010', '01011', '00000', '01011', '01010', '00000', '11000', '01101', '11011', '00101', '00111', '00000', '01010', '01011', '00010', '01000', '00001', '01010', '00000', '00000', '00011', '10011', '00101', '01010', '00000', '00000', '00011', '00111', '00111', '01000', '00101', '00000', '01010', '00001', '00000', '00000', '00000', '01011', '01011', '00000', '11000', '01000', '01000', '00001', '01011', '01010', '01010', '00000', '00011']
> counts: {'01000': 8, '00000': 24, '00001': 9, '01010': 13, '01011': 11, '00100': 3,